In [ ]:
%pylab inline

# JSAnimation import available at https://github.com/jakevdp/JSAnimation
from JSAnimation import IPython_display
from matplotlib import animation

import numpy as np

from ndio.remote.boss.remote import Remote
from ndio.ndresource.boss.resource import *
from ndio.convert.tiff import load_tiff_multipage

import os
from datetime import datetime

import boto3
import requests
from io import BytesIO


## Setup ndio Remote instance for the Boss

We are passing in a config file that is set up with the endpoint location and credentials

In [ ]:
rmt = Remote('demo.cfg')
API_VER = 'v0.5'

## Populating Data via Cutout Service
Let's populated the Boss with some sample data to demonstrate S3 integratino.

We'll start by creating a channel resource for this dataset, which is located in the "brain1" collection, "ac4" experiment, and "em" channel.

Then well pulling down a 1024x1024x64 voxel chunck of EM data from the Kasthuri et. al. 2011 data set from and S3 bucket and write it to the Boss.



In [ ]:
# Create Collection
col = CollectionResource('brain1', API_VER, 'Cutout Example Collection')
try:
    col = rmt.project_create(col)
    if col:
        print("Successfully Created Collection 'brain1'")
except:
    col = rmt.project_get(col)


# Create EM Coordinate Frame
coord_em = CoordinateFrameResource('ac4_coord_frame', API_VER,
                                   'Coordinate frame for AC4 region of EM Data.', 0, 1024, 0, 1024, 0, 64)
try:
    coord_em = rmt.project_create(coord_em)
    if coord_em:
        print("Successfully Created Coordinate Frame 'ac4_coord_frame'")
except:
    # get coordinate props (eventually will fix this so get returns an instance and create returns an ID)
    coord_em = rmt.project_get(coord_em)


# Create Experiment for EM data
em_exp = ExperimentResource('ac4', 'brain1', API_VER, 'Experiment for EM data.', coord_em.id)
try:
    em_exp = rmt.project_create(em_exp)
    if em_exp:
        print("Successfully Created Experiment 'ac4'")
except:
    em_exp = rmt.project_get(em_exp)

em_chan = ChannelResource('em', 'brain1', 'ac4', API_VER, 'EM channel', datatype='uint8')
try:
    em_chan = rmt.project_create(em_chan)
    if em_chan:
        print("Successfully Created Channel 'em'")
except:
    em_chan = rmt.project_get(em_chan)

In [ ]:
# Get numpy array of data to upload that has been previously saved in S3
r = requests.get("https://s3.amazonaws.com/boss-demo-data/kasthuri11_cube.npy")
b = BytesIO(r.content)
data = np.load(b)
print(data.shape)

In [ ]:
# Write data to the boss
x_rng = '0:1024'
y_rng = '0:1024'
z_rng = '0:64'
start = datetime.now()
response = rmt.cutout_create(em_chan, 0, x_rng, y_rng, z_rng, data)
end = datetime.now()
delta = end-start

print("Wrote {}MB to the Boss in {}.{} seconds".format(data.size/10**6, delta.seconds, delta.microseconds))

## Accessing EM Data - uint8 support
We've already populated the Boss with some sample data for this demo.  Let's start by pulling down a 1024x1024x64 voxel chunck of EM data from the Kasthuri et. al. 2011 data set.

Start by creating a channel resource for this dataset, which is located in the "brain1" collection, "ac4" experiment, and "em" channel.

In [ ]:
# Create a resource for the channel you want to access
em_chan = ChannelResource('em', 'brain1', 'ac4', API_VER)

x_rng = '0:1024'
y_rng = '0:1024'
z_rng = '0:64'
print("Downloading data")
start = datetime.now()
data = rmt.cutout_get(em_chan, 0, x_rng, y_rng, z_rng)
end = datetime.now()
delta = end-start
print("Downloaded {}MB from the Boss in {}.{} seconds".format(data.size/10**6, delta.seconds, delta.microseconds))

In [ ]:
# Plot
fig0 = plt.figure(figsize=(6,6))
im = plt.imshow(data[1,:,:], cmap=plt.cm.gray, clim=(data.min(), data.max())) 

def init():
    im.set_data(data[1,:,:])

def animate(i):   
    im.set_data(data[i,:,:])
    return im

animation.FuncAnimation(fig0, animate, init_func=init, frames=data.shape[0],
                               interval=50)

We can get some images from the tile service now

The URL structure is: https://api.theboss.io/v0.5/tiles/(col)/(exp)/(ch)/xy/(res)/(x_range)/(y_range)/(z_slize)

[Slice 0](https://api.theboss.io/v0.5/tiles/brain1/ac4/em/xy/0/0:1024/0:1024/0)

[Slice 1](https://api.theboss.io/v0.5/tiles/brain1/ac4/em/xy/0/0:1024/0:1024/1)

[Slice 2](https://api.theboss.io/v0.5/tiles/brain1/ac4/em/xy/0/0:1024/0:1024/2)

[Slice 50](https://api.theboss.io/v0.5/tiles/brain1/ac4/em/xy/0/0:1024/0:1024/50)

[Smaller Tile](https://api.theboss.io/v0.5/tiles/brain1/ac4/em/xy/0/0:512/0:512/0)